<a href="https://colab.research.google.com/github/vlasova-us/Python_public/blob/main/Parser_from_habr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Функция поиска на сайте habr. В качестве параметра функция принимает список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращает датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>


In [151]:
import time
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np

In [152]:
def get_all_links(query, pages):
  '''
Функция принимает на вход список из поискового запроса(ов) (query) для поиска на сайте https://habr.com , а также глубину поиска (количество поисковых страниц), которую необходимо вывести (pages).
Функция возвращает список из ссылок.

  '''
  url = 'https://habr.com/ru/search/'
  all_links = []
  links_list = []
  params = {'q': query, 'target_type': 'posts','order':'date'}

  for i in range(1, pages+1):
    if i == 1:
      req = requests.get(url, params)
    else: req = requests.get((str(url) + str('page')+str(i)+'/'), params)

    soup = BeautifulSoup (req.text)

    time.sleep(0.33)

    refs = soup.find_all('article', class_='tm-articles-list__item')

    for ref in refs:
      all_links.append(ref.find('a', 'tm-title__link').get('href'))
      all_full_links = list(map(lambda x: 'https://habr.com' + x, all_links))   #Ссылки
      links_list += all_full_links
  return set(links_list)

In [154]:
def search_habr (all_links):
  '''
Функция принимает на вход список из ссылок, полученных функцией get_all_links, от поискового запроса(ов) (query) для поиска на сайте https://habr.com
Функция возвращает DataFrame из столбцов:
- Дата опубликования статьи в datatime формате (колонка date)
- Заголовок статьи (колонка title)
- Ссылка на статью (колонка link)
- Полный текст статьи (колонка text)
- Количество лайков: общее кол-во, позитивные, негативные (колонка like)
  '''
  habr_articles_df=pd.DataFrame()
  for link in all_links:
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.33)

    date = pd.to_datetime (soup.find ('span', class_='tm-article-datetime-published').find('time').get('datetime')) #Дата в формате Datetime

    if soup.find ('h1', class_='tm-title tm-title_h1'): #Заголовок.
     title = soup.find ('h1', class_='tm-title tm-title_h1').find('span').text
    else: title = np.nan

    if soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2'): #Полный текст
      text = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2').text.strip().replace('\n', '')
    elif soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1'):
       text = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1').text.strip().replace('\n', '')
    else: text = np.nan

    row = {'date': date, 'title': title, 'link': link, 'text': text}
    habr_articles_df = pd.concat ([habr_articles_df, pd.DataFrame([row]) ] ) #  Объединение в DataFrame всех строк
    habr_articles_df ['date'] = pd.to_datetime(habr_articles_df ['date']).apply(lambda x: x.date()) #Убираем TimeZone, чтобы прошла выгрузка в Excel



  return habr_articles_df.reset_index (drop = True)


In [158]:
def main ():
  query = input ("Введите запрос: ")
  pages = int(input ("Введите количество страниц поиска: ") )
  all_links = get_all_links(query, pages)
  habr_ = search_habr (all_links)
  # display(habr_)
  habr_.to_excel('./result_search.xlsx')


In [159]:
main()

Введите запрос: dwh
Введите количество страниц поиска: 5
